In [1]:
# # install octave
# !sudo apt-get -qq update
# !sudo apt-get -qq install octave octave-signal liboctave-dev

# # install oct2py that compatible with colab
# import os

# from pkg_resources import get_distribution

# os.system(
#     f"pip install -qq"
#     f" ipykernel=={get_distribution('ipykernel').version}"
#     f" ipython=={get_distribution('ipython').version}"
#     f" tornado=={get_distribution('tornado').version}"
#     f" oct2py"
# )

# # install packages
# !pip install -qq matpower matpowercaseframes

In [2]:
import matpower
import oct2py

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.8.0
matpower version: 8.1.0.2.2.3


In [3]:
import os

import pandas as pd
from matpower import path_matpower, start_instance

In [4]:
path_most_ex_cases = os.path.join(path_matpower, "most/examples")

In [5]:
m = start_instance()

In [6]:
# load single most example case
mpc = m.loadcase(os.path.join(path_most_ex_cases, "ex_case3a.m"))
mpc

{'version': '2',
 'baseMVA': 100.0,
 'bus': array([[  1.  ,   3.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
           0.  , 135.  ,   1.  ,   1.05,   0.95],
        [  2.  ,   2.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
           0.  , 135.  ,   1.  ,   1.05,   0.95],
        [  3.  ,   2.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
           0.  , 135.  ,   1.  ,   1.05,   0.95]]),
 'gen': array([[   1.,  125.,    0.,   25.,  -25.,    1.,  100.,    1.,  200.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  250.,
          250.,    0.,    0.],
        [   1.,  125.,    0.,   25.,  -25.,    1.,  100.,    1.,  200.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  250.,
          250.,    0.,    0.],
        [   2.,  200.,    0.,   50.,  -50.,    1.,  100.,    1.,  500.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  600.,
          600.,    0.,    0.],
        [   3., -450.,    0.,    0.,    0.,    1.,  10

In [7]:
mpc.keys()

dict_keys(['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost', 'reserves'])

In [8]:
from matpowercaseframes.constants import COLUMNS

In [9]:
COLUMNS.keys()

dict_keys(['bus', 'gen', 'branch', 'dcline', 'if', 'gencost', 'dclinecost', 'bus_name', 'branch_name', 'gen_name', 'case'])

In [10]:
mpc_keys = set(mpc.keys())
columns_keys = set(COLUMNS.keys())
missing_keys = mpc_keys - columns_keys
missing_keys

{'baseMVA', 'reserves', 'version'}

In [11]:
mpc.reserves

{'zones': array([[1., 1., 1., 0.]]),
 'req': 150.0,
 'cost': array([[1.],
        [3.],
        [5.]]),
 'qty': array([[100.],
        [100.],
        [200.]])}

Reserve Data

reserve zones, element i, j is 1 if gen j is in zone i, 0 otherwise.
dimension (zone, gen)

```octave
mpc.reserves.zones = [
	1	1	1	0;
];
```

Mapped to `DataFrame` with index follows zone index (from 1), and columns follows
generator index (cf.gen.index).

In [12]:
n_zones, n_gens = mpc.reserves.zones.shape

In [13]:
df_reserves_zone = pd.DataFrame(
    mpc.reserves.zones,
    index=pd.RangeIndex(start=1, stop=n_zones + 1, name="zone"),
    columns=pd.RangeIndex(start=1, stop=n_gens + 1, name="gen"),
)
df_reserves_zone

gen,1,2,3,4
zone,,,,
1,1.0,1.0,1.0,0.0


reserve requirements for each zone in MW

```octave
mpc.reserves.req = [60; 20];
```

or

```octave
mpc.reserves.req = 150;
```

Mapped to `DataFrame` with index follows zone index (from 1), and single column 'PREQ'.

In [14]:
df_reserves_req = pd.DataFrame(
    mpc.reserves.req,
    index=pd.RangeIndex(start=1, stop=n_zones + 1, name="zone"),
    columns=["PREQ"],
)
df_reserves_req

,PREQ
zone,
1,150.0


reserve costs in $/MW for each gen that belongs to at least 1 zone. (same order as gens,
but skipping any gen that does not belong to any zone)

```octave
mpc.reserves.cost  = [	1.9;	2;	3;	4;	5;	5.5	];
```

In [15]:
df_reserves_zone_sum = df_reserves_zone.sum(axis=0)
idx_gen_with_reserves = df_reserves_zone_sum[df_reserves_zone_sum > 0].index
idx_gen_with_reserves

RangeIndex(start=1, stop=4, step=1, name='gen')

In [16]:
df_reserves_cost = pd.DataFrame(
    mpc.reserves.cost, index=idx_gen_with_reserves, columns=["C1"]
)
df_reserves_cost

,C1
gen,
1,1.0
2,3.0
3,5.0


max reserve quantities for each gen that belongs to at least 1 zone. (same order as
gens, but skipping any gen that does not belong to any zone)

```octave
mpc.reserves.qty = [	25;	25;	25;	25;	25;	25	];
```

In [17]:
df_reserves_qty = pd.DataFrame(
    mpc.reserves.qty, index=idx_gen_with_reserves, columns=["PQTY"]
)
df_reserves_qty

,PQTY
gen,
1,100.0
2,100.0
3,200.0


In [25]:
def reserves_to_dataframes(reserves):
    """
    Convert all mpc.reserves data to DataFrames.

    Args:
        reserves: mpc.reserves object from MATPOWER

    Returns:
        Dictionary containing:
            - 'zones': Reserve zones DataFrame
            - 'req': Reserve requirements DataFrame
            - 'cost': Reserve costs DataFrame (if exists)
            - 'qty': Reserve quantities DataFrame (if exists)
    """
    dfs = {}
    n_zones, n_gens = reserves.zones.shape
    dfs["zones"] = pd.DataFrame(
        reserves.zones,
        index=pd.RangeIndex(start=1, stop=n_zones + 1, name="zone"),
        columns=pd.RangeIndex(start=1, stop=n_gens + 1, name="gen"),
    )
    zone_sum = dfs["zones"].sum(axis=0)
    idx_gen_with_reserves = zone_sum[zone_sum > 0].index
    dfs["req"] = pd.DataFrame(
        reserves.req,
        index=pd.RangeIndex(start=1, stop=n_zones + 1, name="zone"),
        columns=["PREQ"],
    )
    if hasattr(reserves, "cost"):
        dfs["cost"] = pd.DataFrame(
            reserves.cost, index=idx_gen_with_reserves, columns=["C1"]
        )
    if hasattr(reserves, "qty"):
        dfs["qty"] = pd.DataFrame(
            reserves.qty, index=idx_gen_with_reserves, columns=["PQTY"]
        )

    return dfs

In [27]:
dfs = reserves_to_dataframes(mpc.reserves)
for key, value in dfs.items():
    print(key)
    display(value)

zones


gen,1,2,3,4,5,6
zone,,,,,,
1,1.0,1.0,1.0,0.0,0.0,0.0


req


,PREQ
zone,
1,150.0


cost


,C1
gen,
1,1.0
2,3.0
3,5.0


qty


,PQTY
gen,
1,100.0
2,100.0
3,200.0


> [!NOTE]  
> Currently, `loadgenericdata` didn't support absolute path. This impacts various `most`
> functions. See: <https://github.com/MATPOWER/matpower/pull/283>.

In [18]:
xgd = m.loadxgendata("ex_xgd_uc.m", mpc)
xgd

{'CommitSched': array([[1.],
        [1.],
        [1.],
        [1.]]),
 'InitialPg': array([[ 125.],
        [ 125.],
        [ 200.],
        [-450.]]),
 'RampWearCostCoeff': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'PositiveActiveReservePrice': array([[5.0e+00],
        [1.0e-08],
        [1.5e+00],
        [1.0e-08]]),
 'PositiveActiveReserveQuantity': array([[250.],
        [250.],
        [600.],
        [800.]]),
 'NegativeActiveReservePrice': array([[1.e+01],
        [2.e-08],
        [3.e+00],
        [2.e-08]]),
 'NegativeActiveReserveQuantity': array([[250.],
        [250.],
        [600.],
        [800.]]),
 'PositiveActiveDeltaPrice': array([[1.e-09],
        [1.e-09],
        [1.e-09],
        [1.e-09]]),
 'NegativeActiveDeltaPrice': array([[1.e-09],
        [1.e-09],
        [1.e-09],
        [1.e-09]]),
 'PositiveLoadFollowReservePrice': array([[1.e-06],
        [1.e-06],
        [1.e+01],
        [1.e-06]]),
 'PositiveLoadFollowReserveQuantity': array

In [19]:
[iwind, mpc, xgd] = m.addwind("ex_wind_uc", mpc, xgd, nout=3)
[iwind, mpc, xgd]

[5.0,
 {'version': '2',
  'baseMVA': 100.0,
  'bus': array([[  1.  ,   3.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
            0.  , 135.  ,   1.  ,   1.05,   0.95],
         [  2.  ,   2.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
            0.  , 135.  ,   1.  ,   1.05,   0.95],
         [  3.  ,   2.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
            0.  , 135.  ,   1.  ,   1.05,   0.95]]),
  'gen': array([[   1.,  125.,    0.,   25.,  -25.,    1.,  100.,    1.,  200.,
             0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  250.,
           250.,    0.,    0.],
         [   1.,  125.,    0.,   25.,  -25.,    1.,  100.,    1.,  200.,
             0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  250.,
           250.,    0.,    0.],
         [   2.,  200.,    0.,   50.,  -50.,    1.,  100.,    1.,  500.,
             0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  600.,
           600.,    0.,    0.],
         [   3., -450.,    0., 

In [20]:
profiles = m.getprofiles("ex_wind_profile_d", iwind)
profiles

{'type': 'mpcData',
 'table': 2.0,
 'rows': 5.0,
 'col': 9.0,
 'chgtype': 2.0,
 'values': array([[0.8 ],
        [0.65],
        [0.6 ],
        [0.82],
        [1.  ],
        [0.7 ],
        [0.5 ],
        [0.85],
        [1.  ],
        [1.1 ],
        [1.06],
        [0.95]])}

In [21]:
profiles = m.getprofiles("ex_load_profile", profiles)
profiles

2x1 StructArray containing the fields:
    type
    table
    rows
    col
    chgtype
    values

In [22]:
# TODO: example on `mdi = loadmd(mpc, transmat, xgd, [], 'ex_contab', profiles);`

In [23]:
[iess, mpc, xgd, sd] = m.addstorage("ex_storage", mpc, xgd, nout=4)
[iess, mpc, xgd, sd]

[6.0,
 {'version': '2',
  'baseMVA': 100.0,
  'bus': array([[  1.  ,   3.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
            0.  , 135.  ,   1.  ,   1.05,   0.95],
         [  2.  ,   2.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
            0.  , 135.  ,   1.  ,   1.05,   0.95],
         [  3.  ,   2.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
            0.  , 135.  ,   1.  ,   1.05,   0.95]]),
  'gen': array([[   1.,  125.,    0.,   25.,  -25.,    1.,  100.,    1.,  200.,
             0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  250.,
           250.,    0.,    0.],
         [   1.,  125.,    0.,   25.,  -25.,    1.,  100.,    1.,  200.,
             0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  250.,
           250.,    0.,    0.],
         [   2.,  200.,    0.,   50.,  -50.,    1.,  100.,    1.,  500.,
             0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  600.,
           600.,    0.,    0.],
         [   3., -450.,    0., 

In [24]:
m.exit()